In [2]:
import pandas as pd
import requests
from joblib import load
from flask import jsonify

In [3]:
model = load('pipeline.pkl') 
url_users = "https://edem-students-backend.vercel.app/users/dataGetAll"

headers = {"Authorization": "desafio2023"}

payload = ""

response = requests.get(url_users,headers=headers, data=payload)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data_users = response.json()  # Assuming the response contains JSON data
    
    # Convert the data to a DataFrame
    users_df = pd.DataFrame(data_users)
else:
    print('Error: Failed to fetch data from the webpage')



In [4]:
users_df.head(3)

,_id,name,surname,email,age,gender,role,roleMde,program,year,connections,categoryIds,eventIds,confirmed,createdAt,updatedAt,__v,image,bio,chatIds
0,64808dcb7ba3ab3822f54b50,Zaira,Verdugo,0zairaverdugo@edem.es,19,female,user,student,64805a42bb868aa40ba43826,4,[],"[{'_id': '64805c99a7607c035063190c', 'name': '...","[6480f7a228734cb852b96b59, 6480f7a228734cb852b...",True,2023-06-07T14:01:48.004Z,2023-06-12T15:51:32.354Z,18,zaira.jpg,Zaira es una chica llena de pasión y determina...,[64873ed6d9de548738fa51c1]
1,64808dcc7ba3ab3822f54b52,Soledad,Palomares,0soledadpalomares@edem.es,18,female,user,student,64805a42bb868aa40ba43826,1,[],"[{'_id': '64805c99a7607c035063190c', 'name': '...",[],True,2023-06-07T14:01:48.020Z,2023-06-12T15:52:28.130Z,2,soledad.jpg,NaN,"[64862fd645764b6443639939, 6486305b9e7e55c61ef..."
2,64808dcc7ba3ab3822f54b54,Jesús,Alberola,0jesúsalberola@edem.es,18,male,user,student,64805a42bb868aa40ba4382d,4,[],"[{'_id': '64805c99a7607c0350631913', 'name': '...",[],True,2023-06-07T14:01:48.033Z,2023-06-07T14:01:48.033Z,0,jesus.jpg,NaN,[]


In [5]:
category_id_list = []
category_name_list = []

# Iterate over each event row
for user in users_df["categoryIds"]:
    category_ids = []
    category_names = []

    # Iterate over each category in the event
    for category in user:
        category_ids.append(category["_id"])
        category_names.append(category["name"])

    # Append the category details to the lists
    category_id_list.append(category_ids)
    category_name_list.append(category_names)

# Assign the category details to the DataFrame
users_df["category_id"] = category_id_list
users_df["category_name"] = category_name_list

In [6]:
url_programs = "https://edem-students-backend.vercel.app/programs/dataGetAll"
headers = {"Authorization": "desafio2023"}
payload = ""

response = requests.get(url_programs,headers=headers, data=payload)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data_programs = response.json()  # Assuming the response contains JSON data
    
    # Convert the data to a DataFrame
    programs_df = pd.DataFrame(data_programs)
else:
    print('Error: Failed to fetch data from the webpage')

# Create a dictionary mapping program IDs to program names
program_dict = dict(zip(programs_df["_id"], programs_df["name"]))

# Use the map function to create the new column "program_name"
users_df["program_name"] = users_df["program"].map(program_dict)


In [7]:
users_id = users_df['_id']
users_df.drop(['role','chatIds','roleMde','program','connections','eventIds','confirmed','createdAt','updatedAt','__v','image','bio','category_id'],1,inplace=True)

C:\Users\Javi\AppData\Local\Temp\ipykernel_22792\3741830297.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  users_df.drop(['role','chatIds','roleMde','program','connections','eventIds','confirmed','createdAt','updatedAt','__v','image','bio','category_id'],1,inplace=True)


In [8]:
users_df.head(2)

,_id,name,surname,email,age,gender,year,categoryIds,category_name,program_name
0,64808dcb7ba3ab3822f54b50,Zaira,Verdugo,0zairaverdugo@edem.es,19,female,4,"[{'_id': '64805c99a7607c035063190c', 'name': '...","[Marketing, Sustainability, Tech, Business Man...",BBA in Business Administration
1,64808dcc7ba3ab3822f54b52,Soledad,Palomares,0soledadpalomares@edem.es,18,female,1,"[{'_id': '64805c99a7607c035063190c', 'name': '...","[Marketing, Finance and Investment]",BBA in Business Administration


In [9]:
users_df.rename(columns={'_id': 'student_id',
                   'categoryIds': 'category_id',
                   'category_name':'category',
                   'program_name':'programme'},
          inplace=True, errors='raise')

In [10]:
# Define the mapping of values to labels
mapping = {'1': '1st year', '2': '2nd year', '3': '3rd year', '4': '4th year'}

# Replace values in the 'year_of_study' column
users_df['year'] = users_df['year'].replace(mapping)

In [11]:
users_df = users_df.dropna()

In [12]:
users_df = model.transform(users_df)

c:\Users\Javi\anaconda3\envs\eda_sec\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
df_final = pd.DataFrame(users_df,columns=['Cluster'])
df_final['_id'] = users_id
df_final

,Cluster,_id
0,11,64808dcb7ba3ab3822f54b50
1,0,64808dcc7ba3ab3822f54b52
2,8,64808dcc7ba3ab3822f54b54
3,10,64808dcc7ba3ab3822f54b56
4,11,64808dcc7ba3ab3822f54b58
5,19,64808dcc7ba3ab3822f54b5a
6,15,64808dcc7ba3ab3822f54b5c
7,10,64808dcc7ba3ab3822f54b5e
8,10,64808dcc7ba3ab3822f54b61
9,17,64808dcc7ba3ab3822f54b63


In [ ]:
# ENDPOINT 6 - Recommend similar users
@app.route('/recommend_users', methods=['GET'])
def recommend_users():
    model = load('pipeline.pkl') 
    url_users = "https://edem-students-backend.vercel.app/users/dataGetAll"
    headers = {"Authorization": "desafio2023"}
    payload = ""
    response = requests.get(url_users,headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data_users = response.json()  # Assuming the response contains JSON data
        # Convert the data to a DataFrame
        users_df = pd.DataFrame(data_users)
    else:
        return 'Error: Failed to fetch data from the webpage'
    
    category_id_list = []
    category_name_list = []

    # Iterate over each event row
    for user in users_df["categoryIds"]:
        category_ids = []
        category_names = []

        # Iterate over each category in the event
        for category in user:
            category_ids.append(category["_id"])
            category_names.append(category["name"])

        # Append the category details to the lists
        category_id_list.append(category_ids)
        category_name_list.append(category_names)

    # Assign the category details to the DataFrame
    users_df["category_id"] = category_id_list
    users_df["category_name"] = category_name_list

    #programs ----------------------------------------------------------
    url_programs = "https://edem-students-backend.vercel.app/programs/dataGetAll"
    headers = {"Authorization": "desafio2023"}
    payload = ""

    response = requests.get(url_programs,headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        data_programs = response.json()  # Assuming the response contains JSON data
        
        # Convert the data to a DataFrame
        programs_df = pd.DataFrame(data_programs)
    else:
        return 'Error: Failed to fetch data from the webpage'

    # Create a dictionary mapping program IDs to program names
    program_dict = dict(zip(programs_df["_id"], programs_df["name"]))

    # Use the map function to create the new column "program_name"
    users_df["program_name"] = users_df["program"].map(program_dict)
    #---------------------------------------------------------------------
    users_id = users_df['_id']
    users_df.drop(['role','chatIds','roleMde','program','connections','eventIds','confirmed','createdAt','updatedAt','__v','image','bio','category_id'],1,inplace=True)
    users_df.rename(columns={'_id': 'student_id','categoryIds': 'category_id','category_name':'category','program_name':'programme'},inplace=True, errors='raise')

    # Define the mapping of values to labels
    mapping = {'1': '1st year', '2': '2nd year', '3': '3rd year', '4': '4th year'}

    # Replace values in the 'year_of_study' column
    users_df['year'] = users_df['year'].replace(mapping)
    users_df = users_df.dropna()
    users_df = model.transform(users_df)
    df_final = pd.DataFrame(users_df,columns=['cluster'])
    df_final['_id'] = users_id
    # Convert DataFrame to JSON
    json_data = df_final.to_json(orient='records')
    # Return JSON response
    return json_data
    